The following tutorial is available on the [Wallaroo Github Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/20231004-wallaroo-inference-server/wallaroo-inference-server-tutorials/wallaroo-inference-server-hf-summarizer).

## Wallaroo Inference Server:  Resnet

This notebook is used in conjunction with the [Wallaroo Inference Server Free Edition](https://docs.wallaroo.ai/wallaroo-inferencing-server/) for the Computer Vision resnet 50 machine learning model.  This provides a free license for performing inferences through the Computer Vision resnet 50 machine learning model.  For full demonstrations of this model, see [Wallaroo Edge Computer Vision Deployment Demonstration](https://docs.wallaroo.ai/20230300/wallaroo-tutorials/wallaroo-edge-publish/wallaroo-edge-cv-deployment-tutorial).

### Prerequisites

* A deployed Wallaroo Inference Server Free Edition with one of the following options:
  * **Wallaroo.AI HF ResNet50 Inference Server - x64**
* Access via port 8080 to the Wallaroo Inference Server Free Edition.

## Wallaroo Inference Server API Endpoints

The following HTTPS API endpoints are available for Wallaroo Inference Server.

### Pipelines Endpoint

* Endpoint: HTTPS GET `/pipelines`
* Returns:
  * List of `pipelines` with the following fields.
    * **id** (*String*): The name of the pipeline.
    * **status** (*String*): The pipeline status.  `Running` indicates the pipeline is available for inferences.

#### Pipeline Endpoint Example

The following demonstrates using `curl` to retrieve the Pipelines endpoint.  Replace the HOSTNAME with the address of your Wallaroo Inference Server.

In [9]:
!curl HOSTNAME:8080/pipelines

{"pipelines":[{"id":"resnet50","status":"Running"}]}

### Models Endpoint

* Endpoint: GET `/models`
* Returns:
  * List of `models` with the following fields.
    * **name** (*String*):  The name of the model.
    * **sha** (*String*):  The `sha` hash of the model.
    * **status** (*String*):  The model status.  `Running` indicates the models is available for inferences.
    * **version** (*String*): The model version in UUID format.

#### Models Endpoint Example

The following demonstrates using `curl` to retrieve the Models endpoint.  Replace the HOSTNAME with the address of your Wallaroo Inference Server.

In [8]:
!curl HOSTNAME:8080/models

{"models":[{"name":"resnet50","sha":"af16a04a6ec48ac494065d4439fe9dea590d337b9ca6dc328160ccf04a217b9c","status":"Running","version":"8131edb2-1ca7-4ef5-bb47-fbc770d60972"}]}

### Inference Endpoint

The inference endpoint takes the following pattern:

* `/pipelines/{pipeline-name}`:  The `pipeline-name` is the same as returned from the [`/pipelines`](#list-pipelines) endpoint as `id`.

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

Once deployed, we can perform an inference through the deployment URL.

The endpoint returns `Content-Type: application/json; format=pandas-records` by default with the following fields:

* **check_failures** (*List[Integer]*): Whether any validation checks were triggered.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing]({{<ref "wallaroo-sdk-essentials-pipeline#anomaly-testing">}}).
* **elapsed** (*List[Integer]*): A list of time in nanoseconds for:
  * [0] The time to serialize the input.
  * [1...n] How long each step took.
* **model_name** (*String*): The name of the model used.
* **model_version** (*String*): The version of the model in UUID format.
* **original_data**: The original input data.  Returns `null` if the input may be too long for a proper return.
* **outputs** (*List*): The outputs of the inference result separated by data type, where each data type includes:
  * **data**: The returned values.
  * **dim** (*List[Integer]*): The dimension shape returned.
  * **v** (*Integer*): The vector shape of the data.
* **pipeline_name**  (*String*): The name of the pipeline.
* **shadow_data**: Any shadow deployed data inferences in the same format as **outputs**.
* **time** (*Integer*): The time since UNIX epoch.


### Inference Endpoint Example

The following example performs an inference using the Apache Arrow table input `./data/image_224x224.arrow` from an image converted into a `tensor` for inferencing.

In [4]:
!curl -X POST martin-freemium-resnet.eastus.cloudapp.azure.com:8080/pipelines/resnet50 \
    -H "Content-Type: application/vnd.apache.arrow.file" \
    --data-binary @./data/image_224x224.arrow

[{"check_failures":[],"elapsed":[77657435,325185470],"model_name":"resnet50","model_version":"8131edb2-1ca7-4ef5-bb47-fbc770d60972","original_data":null,"outputs":[{"Float":{"data":[-0.9209704995155334,-0.10495197772979736,-0.6216869950294495,-0.7639160752296448,-0.2154858112335205,-0.505703330039978,-0.7874149680137634,-0.5596786141395569,-1.1615943908691406,0.568906307220459,0.25151896476745605,-0.4426764249801636,-0.2803022563457489,-0.18332044780254364,-0.5667626857757568,-0.2607719302177429,0.8765640258789062,-0.33152544498443604,-0.09792342782020569,-0.31929707527160645,-0.03542117774486542,2.243022918701172,1.2097502946853638,0.8100910782814026,-0.7545893788337708,-0.5059118866920471,-0.0510006882250309,-0.08611194789409637,0.04777945578098297,-0.9071007370948792,-1.2397105693817139,-0.6117401123046875,-1.2649085521697998,-0.49369877576828003,0.20755505561828613,-1.5142827033996582,-0.7229145765304565,-1.4517813920974731,0.5931597352027893,-0.705878734588623,-0.49715355038642883